# **4-Mer Classifiers**

In [34]:
import pandas as pd


In [35]:
import pickle
X_test = []
with (open("X_test", "rb")) as openfile:
    while True:
        try:
            X_test = pickle.load(openfile)
        except EOFError:
            break
y_test = []
with (open("y_test", "rb")) as openfile:
    while True:
        try:
            y_test = pickle.load(openfile)
        except EOFError:
            break
X_train = []
with (open("X_train", "rb")) as openfile:
    while True:
        try:
            X_train = pickle.load(openfile)
        except EOFError:
            break
y_train = []
with (open("y_train", "rb")) as openfile:
    while True:
        try:
            y_train = pickle.load(openfile)
        except EOFError:
            break

In [36]:
def encodingReverse(guide):
    #this function encodes the guide sequence into a 4x4 matrix
    #varaible decleration
    guideSeq = []
    #for loop to encode the guide sequence
    for i in range(len(guide)):
        if guide[i] == 1:
            guideSeq.append('A')
        elif guide[i] == 2:
            guideSeq.append('C')
        elif guide[i] == 3:
            guideSeq.append('G')
        elif guide[i] == 4:
            guideSeq.append('T')
        else:
            pass
    #return the encoded guide sequence
    return guideSeq

In [37]:
for i in range(len(X_train)):
    X_train[i] = encodingReverse(X_train[i])
for i in range(len(X_test)):
    X_test[i] = encodingReverse(X_test[i])

In [38]:
def count_kmers(read, k):
    """Count kmer occurrences in a given read.

    Parameters
    ----------
    read : string
        A single DNA sequence.
    k : int
        The value of k for which to count kmers.

    Returns
    -------
    counts : dictionary, {'string': int}
        A dictionary of counts keyed by their individual kmers (strings
        of length k).

    Examples
    --------
    >>> count_kmers("GATGAT", 3)
    {'ATG': 1, 'GAT': 2, 'TGA': 1}
    """
    # Start with an empty dictionary
    counts = {}
    # Calculate how many kmers of length k there are
    num_kmers = len(read) - k + 1
    # Loop over the kmer start positions
    for i in range(num_kmers):
        # Slice the string to get the kmer
        kmer = read[i:i+k]
        # Add the kmer to the dictionary if it's not there
        if kmer not in counts:
            counts[kmer] = 0
        # Increment the count for this kmer
        counts[kmer] += 1
    # Return the final counts
    return counts

In [39]:
#Emx1 genome 
EMX1_GENOME = r"/Users/brendanwallace-nash/Library/CloudStorage/OneDrive-QueenslandUniversityofTechnology/Research Project-Brendan’s MacBook Pro/Guide Sequence/EMX1.fna"

In [40]:
#get the genome string for EMX1, this will be used to get all 64 lenght 3 kmers
seq = ""
guideSeq = {}
with open(EMX1_GENOME, 'r') as fp:
    for line in fp:
        if not line[0] == ">":
            seq += line.rstrip()

In [41]:
#this is to get a dictionary of all 64 kmers in a dictioanry and set the value to 0
kmerDic = count_kmers(seq, 4)
for key in kmerDic:
    kmerDic[key] = 0

In [42]:
def getKmers(guideList, empKemerDic):
    #this function takes in a list of guide sequences and a dictionary of all 64 kmers and returns a list of dictionaries
    #variable decleration
    guideKmerList = []
    finalKmerList = []
    #for loop to get the kmers for each guide sequence
    for i in range(len(guideList)):
        string = ""
        for letter in X_train[i]:
            string = string + letter
        string = string.upper()
        guideKmerList.append(count_kmers(string, 4))
    #for loop to add the kmers to the dictionary
    for i in range(len(guideKmerList)):
        tempKmerDic = empKemerDic.copy()
        for key in guideKmerList[i]:
            tempKmerDic[key] += guideKmerList[i][key]
        finalKmerList.append(tempKmerDic)
    #return the dictionary
    return finalKmerList

In [43]:
#Convert the guide sequences into a list of dictionaries
trainKmers = getKmers(X_train, kmerDic)
testKmers = getKmers(X_test, kmerDic)

In [44]:
print(len(trainKmers))
print(len(testKmers))

556
62


In [45]:
trainKmersDF = pd.DataFrame(trainKmers)
testKmersDF = pd.DataFrame(testKmers)


In [46]:
trainKmersDF

,GAGC,AGCG,GCGC,CGCT,GCTC,CTCT,TCTC,CTCC,TCCA,CCAA,...,TGTA,ATCA,TAGT,CTAT,TATG,GATA,TATC,ATTG,TCAT,GTAT
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
552,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
553,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
554,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
for c in trainKmersDF.columns:
    print("----{}----".format(c))
    print(trainKmersDF[c].value_counts())

----GAGC----
0    500
1     51
2      5
Name: GAGC, dtype: int64
----AGCG----
0    529
1     25
2      2
Name: AGCG, dtype: int64
----GCGC----
0    505
1     47
2      4
Name: GCGC, dtype: int64
----CGCT----
0    542
1     14
Name: CGCT, dtype: int64
----GCTC----
0    513
1     42
2      1
Name: GCTC, dtype: int64
----CTCT----
0    520
1     34
2      2
Name: CTCT, dtype: int64
----TCTC----
0    518
1     37
2      1
Name: TCTC, dtype: int64
----CTCC----
0    508
1     44
2      4
Name: CTCC, dtype: int64
----TCCA----
0    517
1     37
2      2
Name: TCCA, dtype: int64
----CCAA----
0    520
1     34
2      2
Name: CCAA, dtype: int64
----CAAA----
0    519
1     34
2      3
Name: CAAA, dtype: int64
----AAAC----
0    521
1     35
Name: AAAC, dtype: int64
----AACC----
0    521
1     32
2      3
Name: AACC, dtype: int64
----ACCA----
0    531
1     24
2      1
Name: ACCA, dtype: int64
----CCAC----
0    501
1     52
2      3
Name: CCAC, dtype: int64
----CACG----
0    511
1     44
2      1
Nam

In [48]:
testKmersDF

,GAGC,AGCG,GCGC,CGCT,GCTC,CTCT,TCTC,CTCC,TCCA,CCAA,...,TGTA,ATCA,TAGT,CTAT,TATG,GATA,TATC,ATTG,TCAT,GTAT
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
def dfValueToList(df):
    #this function takes in a dataframe and returns a list 
    #variable decleration
    kmerList = []
    #for loop to convert the dataframe to a list 
    for i in range(len(df)):
            kmerList.append(list(df.iloc[i]))
    #return the list
    return kmerList

In [50]:
X_train = dfValueToList(trainKmersDF)
X_test = dfValueToList(testKmersDF)

# **Build Classification models**

In [51]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score


**K nearest neighbors**

In [52]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3) # Define classifier
knn.fit(X_train, y_train) # Train model

# Make predictions
y_train_pred = knn.predict(X_train)
y_test_pred = knn.predict(X_test)

# Training set performance
knn_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
knn_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
knn_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
knn_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
knn_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
knn_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
knn_test_ROC = roc_auc_score(y_test, y_test_pred) # Calculate ROC AUC
knn_test_recall = recall_score(y_test, y_test_pred, average='weighted') # Calculate Recall
knn_test_precision = precision_score(y_test, y_test_pred, average='weighted') # Calculate Precision

print('Model performance for Training set')
print('- Accuracy: %s' % knn_train_accuracy)
print('- MCC: %s' % knn_train_mcc)
print('- F1 score: %s' % knn_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % knn_test_accuracy)
print('- MCC: %s' % knn_test_mcc)
print('- F1 score: %s' % knn_test_f1)
print('- ROC AUC: %s' % knn_test_ROC)
print('- Recall: %s' % knn_test_recall)
print('- Precision: %s' % knn_test_precision)

Model performance for Training set
- Accuracy: 0.8938848920863309
- MCC: 0.7878807690550698
- F1 score: 0.8938728759589073
----------------------------------
Model performance for Test set
- Accuracy: 0.5645161290322581
- MCC: 0.13145556003198527
- F1 score: 0.5641761728488327
- ROC AUC: 0.565625
- Recall: 0.5645161290322581
- Precision: 0.5664880169885731


In [73]:
#save model as a pickle file
filname = 'knn_model4Mer.sav'
pickle.dump(knn, open(filname, 'wb'))

**Support vector machine (Radial basis function kernel)**

In [53]:
from sklearn.svm import SVC

svm_rbf = SVC(gamma=100, C=100)
svm_rbf.fit(X_train, y_train)

# Make predictions
y_train_pred = svm_rbf.predict(X_train)
y_test_pred = svm_rbf.predict(X_test)

# Training set performance
svm_rbf_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
svm_rbf_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
svm_rbf_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
svm_rbf_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
svm_rbf_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
svm_rbf_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
svm_rbf_test_ROC = roc_auc_score(y_test, y_test_pred) # Calculate ROC AUC
svm_rbf_test_recall = recall_score(y_test, y_test_pred, average='weighted') # Calculate Recall
svm_rbf_test_precision = precision_score(y_test, y_test_pred, average='weighted') # Calculate Precision

print('Model performance for Training set')
print('- Accuracy: %s' % svm_rbf_train_accuracy)
print('- MCC: %s' % svm_rbf_train_mcc)
print('- F1 score: %s' % svm_rbf_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % svm_rbf_test_accuracy)
print('- MCC: %s' % svm_rbf_test_mcc)
print('- F1 score: %s' % svm_rbf_test_f1)
print('- ROC AUC: %s' % svm_rbf_test_ROC)
print('- Recall: %s' % svm_rbf_test_recall)
print('- Precision: %s' % svm_rbf_test_precision)

Model performance for Training set
- Accuracy: 1.0
- MCC: 1.0
- F1 score: 1.0
----------------------------------
Model performance for Test set
- Accuracy: 0.5645161290322581
- MCC: 0.1272823676500175
- F1 score: 0.5641754636833916
- ROC AUC: 0.5635416666666666
- Recall: 0.5645161290322581
- Precision: 0.5641284929382816


In [74]:
#save model as a pickle file
filname = 'svm_rbf_model4Mer.sav'
pickle.dump(svm_rbf, open(filname, 'wb'))

**SVM Polynomial**

In [54]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

poly_kernel_svm_clf = Pipeline((
("scaler", StandardScaler()),
("svm_clf", SVC(kernel="poly", degree=17, coef0=0, C=1000))
))
poly_kernel_svm_clf.fit(X_train, y_train) # Train model

# Make predictions
y_train_pred = poly_kernel_svm_clf.predict(X_train)
y_test_pred = poly_kernel_svm_clf.predict(X_test)

# Training set performance
dt_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
dt_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
dt_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
dt_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
dt_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
dt_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
dt_test_ROC = roc_auc_score(y_test, y_test_pred) # Calculate ROC AUC
dt_test_recall = recall_score(y_test, y_test_pred, average='weighted') # Calculate Recall
dt_test_precision = precision_score(y_test, y_test_pred, average='weighted') # Calculate Precision

print('Model performance for Training set')
print('- Accuracy: %s' % dt_train_accuracy)
print('- MCC: %s' % dt_train_mcc)
print('- F1 score: %s' % dt_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % dt_test_accuracy)
print('- MCC: %s' % dt_test_mcc)
print('- F1 score: %s' % dt_test_f1)
print('- ROC AUC: %s' % dt_test_ROC)
print('- Recall: %s' % dt_test_recall)
print('- Precision: %s' % dt_test_precision)

Model performance for Training set
- Accuracy: 0.9622302158273381
- MCC: 0.9270893079081136
- F1 score: 0.9621711024379613
----------------------------------
Model performance for Test set
- Accuracy: 0.5483870967741935
- MCC: 0.09791666666666667
- F1 score: 0.5483870967741935
- ROC AUC: 0.5489583333333333
- Recall: 0.5483870967741935
- Precision: 0.5495295698924731


In [75]:
#save model as a pickle file
filname = 'poly_kernel_svm_clf_model4Mer.sav'
pickle.dump(poly_kernel_svm_clf, open(filname, 'wb'))

**Random forest**

In [55]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=1000) # Define classifier
rf.fit(X_train, y_train) # Train model

# Make predictions
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

# Training set performance
rf_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
rf_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
rf_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
rf_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
rf_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
rf_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
rf_test_ROC = roc_auc_score(y_test, y_test_pred) # Calculate ROC AUC
rf_test_recall = recall_score(y_test, y_test_pred, average='weighted') # Calculate Recall
rf_test_precision = precision_score(y_test, y_test_pred, average='weighted') # Calculate Precision

print('Model performance for Training set')
print('- Accuracy: %s' % rf_train_accuracy)
print('- MCC: %s' % rf_train_mcc)
print('- F1 score: %s' % rf_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % rf_test_accuracy)
print('- MCC: %s' % rf_test_mcc)
print('- F1 score: %s' % rf_test_f1)
print('- ROC AUC: %s' % rf_test_ROC)
print('- Recall: %s' % rf_test_recall)
print('- Precision: %s' % rf_test_precision)

Model performance for Training set
- Accuracy: 1.0
- MCC: 1.0
- F1 score: 1.0
----------------------------------
Model performance for Test set
- Accuracy: 0.5645161290322581
- MCC: 0.1272823676500175
- F1 score: 0.5641754636833916
- ROC AUC: 0.5635416666666666
- Recall: 0.5645161290322581
- Precision: 0.5641284929382816


In [76]:
#save model as a pickle file
filname = 'rf_model4Mer.sav'
pickle.dump(rf, open(filname, 'wb'))

**Multi-Layer Perceptron**

In [56]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(alpha=0.01, max_iter=10)
mlp.fit(X_train, y_train)

# Make predictions
y_train_pred = mlp.predict(X_train)
y_test_pred = mlp.predict(X_test)

# Training set performance
mlp_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
mlp_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
mlp_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
mlp_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
mlp_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
mlp_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
mlp_test_ROC = roc_auc_score(y_test, y_test_pred) # Calculate ROC AUC
mlp_test_recall = recall_score(y_test, y_test_pred, average='weighted') # Calculate Recall
mlp_test_precision = precision_score(y_test, y_test_pred, average='weighted') # Calculate Precision


print('Model performance for Training set')
print('- Accuracy: %s' % mlp_train_accuracy)
print('- MCC: %s' % mlp_train_mcc)
print('- F1 score: %s' % mlp_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % mlp_test_accuracy)
print('- MCC: %s' % mlp_test_mcc)
print('- F1 score: %s' % mlp_test_f1)
print('- ROC AUC: %s' % mlp_test_ROC)
print('- Recall: %s' % mlp_test_recall)
print('- Precision: %s' % mlp_test_precision)

Model performance for Training set
- Accuracy: 0.8633093525179856
- MCC: 0.7285728581196556
- F1 score: 0.8631500043284933
----------------------------------
Model performance for Test set
- Accuracy: 0.6290322580645161
- MCC: 0.2568081977866954
- F1 score: 0.6256109481915934
- ROC AUC: 0.6260416666666666
- Recall: 0.6290322580645161
- Precision: 0.630514385353095


/Users/brendanwallace-nash/tensorflow-test/env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


In [77]:
#save model as a pickle file
filname = 'mlp_model4Mer.sav'
pickle.dump(mlp, open(filname, 'wb'))


# **Build Stacked model**

In [72]:
# Define estimators
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

estimator_list = [
    ('knn',knn),
    ('svm_rbf',svm_rbf),
    ('poly_kernel_svm_clf',poly_kernel_svm_clf),
    ('rf',rf),
    ('mlp',mlp), ]

# Build stack model
stack_model = StackingClassifier(
    estimators=estimator_list, final_estimator=LogisticRegression()
    )

# Train stacked model
stack_model.fit(X_train, y_train)

# Make predictions
y_train_pred = stack_model.predict(X_train)
y_test_pred = stack_model.predict(X_test)

# Training set model performance
stack_model_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
stack_model_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
stack_model_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set model performance
stack_model_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
stack_model_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
stack_model_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
stack_model_test_ROC = roc_auc_score(y_test, y_test_pred) # Calculate ROC AUC
stack_model_test_recall = recall_score(y_test, y_test_pred, average='weighted') # Calculate Recall
stack_model_test_precision = precision_score(y_test, y_test_pred, average='weighted') # Calculate Precision

print('Model performance for Training set')
print('- Accuracy: %s' % stack_model_train_accuracy)
print('- MCC: %s' % stack_model_train_mcc)
print('- F1 score: %s' % stack_model_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % stack_model_test_accuracy)
print('- MCC: %s' % stack_model_test_mcc)
print('- F1 score: %s' % stack_model_test_f1)
print('- ROC AUC: %s' % stack_model_test_ROC)
print('- Recall: %s' % stack_model_test_recall)
print('- Precision: %s' % stack_model_test_precision)

/Users/brendanwallace-nash/tensorflow-test/env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/brendanwallace-nash/tensorflow-test/env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/brendanwallace-nash/tensorflow-test/env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/brendanwallace-nash/tensorflow-test/env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the 

Model performance for Training set
- Accuracy: 1.0
- MCC: 1.0
- F1 score: 1.0
----------------------------------
Model performance for Test set
- Accuracy: 0.5645161290322581
- MCC: 0.1272823676500175
- F1 score: 0.5641754636833916
- ROC AUC: 0.5635416666666666
- Recall: 0.5645161290322581
- Precision: 0.5641284929382816


In [79]:
#save model as a pickle file
filname = 'stack_model14Mer.sav'
pickle.dump(stack_model, open(filname, 'wb'))

In [58]:
# Define estimators
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

estimator_list = [
    ('knn',knn),
    ('poly_kernel_svm_clf',poly_kernel_svm_clf),
    ('rf',rf) ]

# Build stack model
stack_model = StackingClassifier(
    estimators=estimator_list, final_estimator=LogisticRegression()
)

# Train stacked model
stack_model.fit(X_train, y_train)

# Make predictions
y_train_pred = stack_model.predict(X_train)
y_test_pred = stack_model.predict(X_test)

# Training set model performance
stack_model_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
stack_model_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
stack_model_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set model performance
stack_model_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
stack_model_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
stack_model_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
stack_model_test_ROC = roc_auc_score(y_test, y_test_pred) # Calculate ROC AUC
stack_model_test_recall = recall_score(y_test, y_test_pred, average='weighted') # Calculate Recall
stack_model_test_precision = precision_score(y_test, y_test_pred, average='weighted') # Calculate Precision

print('Model performance for Training set')
print('- Accuracy: %s' % stack_model_train_accuracy)
print('- MCC: %s' % stack_model_train_mcc)
print('- F1 score: %s' % stack_model_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % stack_model_test_accuracy)
print('- MCC: %s' % stack_model_test_mcc)
print('- F1 score: %s' % stack_model_test_f1)
print('- ROC AUC: %s' % stack_model_test_ROC)
print('- Recall: %s' % stack_model_test_recall)
print('- Precision: %s' % stack_model_test_precision)

Model performance for Training set
- Accuracy: 1.0
- MCC: 1.0
- F1 score: 1.0
----------------------------------
Model performance for Test set
- Accuracy: 0.5645161290322581
- MCC: 0.1272823676500175
- F1 score: 0.5641754636833916
- ROC AUC: 0.5635416666666666
- Recall: 0.5645161290322581
- Precision: 0.5641284929382816


In [80]:
#save model as a pickle file
filname = 'stack_model24Mer.sav'
pickle.dump(stack_model, open(filname, 'wb'))

In [59]:
# Define estimators
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

estimator_list = [
    ('knn',knn),
    ('svm_rbf',svm_rbf),
    ('poly_kernel_svm_clf',poly_kernel_svm_clf),
    ('rf',rf) ]

# Build stack model
stack_model = StackingClassifier(
    estimators=estimator_list, final_estimator=LogisticRegression()
)

# Train stacked model
stack_model.fit(X_train, y_train)

# Make predictions
y_train_pred = stack_model.predict(X_train)
y_test_pred = stack_model.predict(X_test)

# Training set model performance
stack_model_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
stack_model_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
stack_model_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set model performance
stack_model_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
stack_model_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
stack_model_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
stack_model_test_ROC = roc_auc_score(y_test, y_test_pred) # Calculate ROC AUC
stack_model_test_recall = recall_score(y_test, y_test_pred, average='weighted') # Calculate Recall
stack_model_test_precision = precision_score(y_test, y_test_pred, average='weighted') # Calculate Precision

print('Model performance for Training set')
print('- Accuracy: %s' % stack_model_train_accuracy)
print('- MCC: %s' % stack_model_train_mcc)
print('- F1 score: %s' % stack_model_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % stack_model_test_accuracy)
print('- MCC: %s' % stack_model_test_mcc)
print('- F1 score: %s' % stack_model_test_f1)
print('- ROC AUC: %s' % stack_model_test_ROC)
print('- Recall: %s' % stack_model_test_recall)
print('- Precision: %s' % stack_model_test_precision)

Model performance for Training set
- Accuracy: 1.0
- MCC: 1.0
- F1 score: 1.0
----------------------------------
Model performance for Test set
- Accuracy: 0.5645161290322581
- MCC: 0.1272823676500175
- F1 score: 0.5641754636833916
- ROC AUC: 0.5635416666666666
- Recall: 0.5645161290322581
- Precision: 0.5641284929382816


In [81]:
#save model as a pickle file
filname = 'stack_model34Mer.sav'
pickle.dump(stack_model, open(filname, 'wb'))